In [1]:
import pandas as pd
import polars as pl
import numpy as np
import os
import sys
from datetime import datetime
import winsound

# Adicionando caminhos dos módulos (ajuste se necessário)
sys.path.insert(0, r'C:\Scripts\modules_azure\database')
sys.path.insert(0, r'C:\Scripts\modules_azure\parameters')

# Importando os módulos novos
from connection_azure import Connect
from azure_loader import AzureLoader
from parametros import Parametros


SCHEMA_DEFAULT = "dbo" 
CAMINHO_RELATORIOS = Parametros.caminho_completo_atual()

<h3>Definição dos parametros</h3>

In [2]:
if not CAMINHO_RELATORIOS:
    print("Erro: Pasta de relatórios de hoje não encontrada.")
    sys.exit()

print(f"Diretório base: {CAMINHO_RELATORIOS}")

Diretório base: C:\Scripts\relatórios\\05-02-2026\0901\


<h1><center>Migração das tabelas</center></h1>

#####  Tabela times

In [3]:
print("Subindo a tabela de Times")
try:
    times = pl.read_excel("C:\\Scripts\\times_atria.xlsx")
    times = times.with_columns(pl.col("Assessor").str.to_uppercase())
    
    # Envio com AzureLoader
    AzureLoader.enviar_df(
        df=times.to_pandas(),
        nome_tabela="times_nova_empresa",
        col_pk="Assessor",
        if_exists="replace",
        schema=SCHEMA_DEFAULT
    )
except Exception as e:
    print(f"Erro em Times: {e}")

Subindo a tabela de Times
[AzureLoader] Subindo tabela: dbo.times_nova_empresa (15 linhas)...
[AzureLoader] Chunksize calculado: 696 linhas por lote (para 3 colunas).
[AzureLoader] Configurando Primary Key: Assessor
[AzureLoader] Concluido: times_nova_empresa atualizada.


##### Tipo clientes

In [6]:
print("Classificando Tipo de Clientes")
try:
    # Lê as tabelas atuais do Azure para comparação
    base_btg_db = AzureLoader.ler_tabela("base_btg", schema=SCHEMA_DEFAULT)
    tipo_clientes_hist = AzureLoader.ler_tabela("tipo_clientes", schema=SCHEMA_DEFAULT)
    
    if not base_btg_db.empty:
        tipo_clientes = base_btg_db.loc[:, ["Conta", "Tipo"]]
        
        # Filtra apenas contas que NÃO estão no histórico
        if not tipo_clientes_hist.empty:
            tipo_clientes = tipo_clientes[~tipo_clientes['Conta'].isin(tipo_clientes_hist['Conta'])]
        
        if not tipo_clientes.empty:
            tipo_clientes.loc[tipo_clientes['Tipo'].isnull(), "Tipo"] = 'Offshore'
            
            # Append apenas dos novos
            AzureLoader.enviar_df(
                df=tipo_clientes,
                nome_tabela="tipo_clientes",
                if_exists="append", # Mantém o histórico, adiciona novos
                schema=SCHEMA_DEFAULT
                # Nota: Não passamos col_pk aqui pois é append, a tabela já existe
            )
        else:
            print("Nenhum cliente novo para classificar.")
    else:
        print("Tabela base_btg vazia ou inexistente no banco.")

except Exception as e:
    print(f"Erro em Tipo Clientes: {e}")

Classificando Tipo de Clientes
[AzureLoader] Lendo: dbo.base_btg...
[AzureLoader] Lendo: dbo.tipo_clientes...
[AzureLoader] Subindo tabela: dbo.tipo_clientes (1 linhas)...
[AzureLoader] Chunksize calculado: 1045 linhas por lote (para 2 colunas).
[AzureLoader] Concluido: tipo_clientes atualizada.


##### Base BTG

In [5]:
print("Atualizando Base BTG e Offshore")
try:
    # Leitura do Excel do dia
    base = pd.read_excel(os.path.join(CAMINHO_RELATORIOS, "base_btg.xlsx"), header=2)
    base.rename(columns={"E-Mail Comunicação": "E-mail"}, inplace=True)
    
    # Limpeza de nomes de colunas (R$)
    base.columns = [col.replace(" (R$)", "") for col in base.columns]
    base['Conta'] = base['Conta'].astype(str).str.strip()
    
    # Tratamentos de texto
    base['Assessor'] = base['Assessor'].astype(str).str.upper()
    
    # Normalização Faixa Cliente
    base.loc[base['Faixa Cliente'].isin(["Ate 50K", "Entre 50k e 100k", "Entre 100k e 300k"]), "Faixa Cliente"] = "Até 300k"
    
    # Processamento Offshore
    offshore = pd.read_excel(r"C:\Scripts\historico_auc\AuC Offshore.xlsx", sheet_name='AuC Offshore')
    offshore = offshore.loc[:, ["Nome", "Conta", "AUC BRL", "Assessor"]]
    offshore.rename(columns={"AUC BRL": "PL Total"}, inplace=True)
    offshore['Conta'] = offshore['Conta'].astype(str).str.strip()
    offshore['Assessor'] = offshore['Assessor'].astype(str).str.upper()
    
    # Upload PL Offshore
    AzureLoader.enviar_df(
        df=offshore,
        nome_tabela="pl_offshore",
        col_pk="Conta",
        if_exists="replace",
        schema=SCHEMA_DEFAULT
    )
    
    # Concatenação e Ajustes Finais Base
    base_full = pd.concat([offshore, base], axis=0)
    
    # Ajuste Assessores Reais
    base_full.loc[base_full['Assessor'] == "MURILO LUIZ SILVA GINO", "Assessor"] = "IZADORA VILLELA FREITAS"
    base_full.loc[base_full['Assessor'].str.contains("GABRIEL GUERRERO TORRES FONSECA", na=False), "Assessor"] = "MARCOS SOARES PEREIRA FILHO"
    
    base_full.drop_duplicates(subset="Conta", keep='first', inplace=True)
    base_full.rename(columns={"Perfil de Cliente": "Perfil do Cliente"}, inplace=True)
    
    # Ajuste Rodrigo
    base_full.loc[base_full['Assessor'].isin(["Rodrigo de Mello D?Elia", "Rodrigo de Mello DElia", "RODRIGO DE MELLO DELIA"]), "Assessor"] = "RODRIGO DE MELLO D’ELIA"
    
    # Upload Base BTG Completa
    AzureLoader.enviar_df(
        df=base_full,
        nome_tabela="base_btg",
        col_pk="Conta",
        if_exists="replace",
        schema=SCHEMA_DEFAULT
    )

except Exception as e:
    print(f"Erro em Base BTG/Offshore: {e}")

Atualizando Base BTG e Offshore
[AzureLoader] Subindo tabela: dbo.pl_offshore (57 linhas)...
[AzureLoader] Chunksize calculado: 522 linhas por lote (para 4 colunas).
[AzureLoader] Configurando Primary Key: Conta
[AzureLoader] Concluido: pl_offshore atualizada.
[AzureLoader] Subindo tabela: dbo.base_btg (1243 linhas)...
[AzureLoader] Chunksize calculado: 46 linhas por lote (para 45 colunas).
[AzureLoader] Configurando Primary Key: Conta
[AzureLoader] Concluido: base_btg atualizada.


In [ ]:
base['Assessor'] = base['Assessor'].apply(lambda nome: str(nome).upper())
base.loc[base['Faixa Cliente'] == "Ate 50K", "Faixa Cliente"] = "Até 300k"
base.loc[base['Faixa Cliente'] == "Entre 50k e 100k", "Faixa Cliente"] = "Até 300k"
base.loc[base['Faixa Cliente'] == "Entre 100k e 300k", "Faixa Cliente"] = "Até 300k"

In [ ]:
offshore = pd.read_excel(r"C:\Scripts\historico_auc\AuC Offshore.xlsx", sheet_name='AuC Offshore')
offshore = offshore.loc[:,["Nome", "Conta", "AUC BRL", "Assessor"]]
offshore.rename(columns={"AUC BRL":"PL Total"}, inplace=True)
offshore['Conta'] = offshore['Conta'].astype(str)
offshore['Assessor'] = [nome.upper() for nome in offshore['Assessor']]

In [ ]:
print(offshore.head(20)) # Mostra as 20 primeiras linhas

In [ ]:

conexao = Connect.connect_techdb()
offshore.to_sql(
    "pl_offshore",
    con=conexao,
    index=False,
    if_exists="replace",
    dtype={"Conta": sqlalchemy.VARCHAR(255)},
    schema="principal"
)

conexao.execute(
    text('ALTER TABLE principal."pl_offshore" ADD PRIMARY KEY ("Conta")')
)

conexao.close()

In [ ]:
base = pd.concat([offshore, base], axis=0)

In [ ]:
#### Ajuste assessores reais
base.loc[base['Assessor'] == "MURILO LUIZ SILVA GINO", "Assessor"] = "IZADORA VILLELA FREITAS"
base.loc[base['Assessor'].str.contains("GABRIEL GUERRERO TORRES FONSECA"), "Assessor"] = "MARCOS SOARES PEREIRA FILHO"

In [ ]:
base.drop_duplicates(subset="Conta", keep='first', inplace=True)
base.rename(columns={"Perfil de Cliente":"Perfil do Cliente"}, inplace=True)

In [ ]:
### Ajuste nome do rodrigo

base.loc[base['Assessor'] == "Rodrigo de Mello D?Elia", "Assessor"] = "RODRIGO DE MELLO D’ELIA"
base.loc[base['Assessor'] == "Rodrigo de Mello DElia", "Assessor"] = "RODRIGO DE MELLO D’ELIA"
base.loc[base['Assessor'] == "RODRIGO DE MELLO DELIA", "Assessor"] = "RODRIGO DE MELLO D’ELIA"

In [ ]:
conexao = Connect.connect_techdb()
base.to_sql("base_btg",
            con=conexao, 
            index=False, 
            if_exists="replace", 
            schema="principal",
            dtype={"Conta":sqlalchemy.VARCHAR(255)})
conexao.execute(text('ALTER TABLE principal."base_btg" ADD PRIMARY KEY ("Conta")'))
conexao.close()

#### Acompanhamento de custódia


In [ ]:
files_diretorio_base = os.listdir(diretorio_base+"%s" % (data))
files_diretorio_base = [element for element in files_diretorio_base if 'acompanhamento-de-custodia' in element]
files_diretorio_base = files_diretorio_base[0]

acompanhamento_de_custodia = pd.read_excel(diretorio_base+"%s\\%s" % (data, files_diretorio_base))

In [ ]:
conexao = Connect.connect_techdb()
acompanhamento_de_custodia.to_sql('acompanhamento_custodia', 
                                    con=conexao, 
                                    index=False, 
                                    if_exists='replace',
                                    schema="principal")
conexao.close()

##### Nomes completos

In [ ]:
nomes_clientes = pd.read_excel(r"C:\Scripts\nomes_clientes\Nomes_Clientes2.xlsx")
nomes_clientes.rename(columns={"Código":"Conta"}, inplace=True)
nomes_clientes['Conta'] = nomes_clientes['Conta'].astype(str)
nomes_clientes = nomes_clientes[~nomes_clientes['Nome'].isna()]

In [ ]:
base_btg = base[['Conta']]
nomes_clientes = pd.merge(base_btg, nomes_clientes, on="Conta", how='outer')

In [ ]:
for conta_sem_nome in nomes_clientes[nomes_clientes['Nome'].isnull()]['Conta']:
    try:
        nomes_clientes.loc[nomes_clientes['Conta'] == conta_sem_nome, 'Nome'] = \
            base[base['Conta'] == conta_sem_nome]['Nome'].values[0]
    except:
        pass

In [ ]:
nomes_clientes['Conta'] = nomes_clientes['Conta'].astype(str)
nomes_clientes.drop_duplicates(subset='Conta', inplace=True)

In [ ]:
nomes_clientes.to_excel(r"C:\Scripts\nomes_clientes\Nomes_Clientes2.xlsx", header=True, index=False)
conexao = Connect.connect_techdb()
nomes_clientes.to_sql('nomes_clientes', 
                      con=conexao, 
                      index=False, 
                      if_exists='replace',
                      schema="principal")
conexao.execute(text('ALTER TABLE principal."nomes_clientes" ADD PRIMARY KEY ("Conta")'))
conexao.close()

<h3>Saldo CC</h3>

In [ ]:
saldo_cc = pd.read_excel(diretorio_base+"%s\\%s\\saldo_cc.xlsx" % (data, horario), header=2)
for columns in saldo_cc.columns:
    saldo_cc.rename(columns={columns:columns.replace(" (R$)","")}, inplace=True)
saldo_cc.rename(columns={"Saldo":"SALDO"}, inplace=True)
saldo_cc = saldo_cc.loc[:,["Conta", "SALDO"]]
saldo_cc['Conta'] = saldo_cc['Conta'].astype(str)
saldo_cc = pd.merge(saldo_cc, base[['Assessor', 'Conta']], on='Conta', how='left')

conexao = Connect.connect_techdb()
saldo_cc.to_sql("saldo_conta_corrente",
                con=conexao, 
                index=False, 
                if_exists='replace', 
                dtype={"Conta":sqlalchemy.VARCHAR(255)},
                schema="principal")
conexao.execute(text('ALTER TABLE principal."saldo_conta_corrente" ADD PRIMARY KEY ("Conta")'))
conexao.close()

<h2>Posição</h2>

In [ ]:
posicao = pd.read_excel(diretorio_base+"%s\\%s\posição.xlsx" % (data, horario), header=2)
for columns in posicao.columns:
    posicao.rename(columns={columns:columns.replace(" (R$)","")}, inplace=True)
posicao.rename(columns={'CONTA':'Conta', 'Submercado':'Sub Mercado'}, inplace=True)
posicao['Conta'] = posicao['Conta'].astype(str)
posicao.rename(columns={"Vencimento":"VENCIMENTO", "Escritório":"ESCRITÓRIO"}, inplace=True)
posicao['VENCIMENTO'] = pd.to_datetime(posicao['VENCIMENTO'], format="%Y-%m-%d", errors='coerce')
posicao = pd.merge(posicao, base[['Conta','Assessor']], on='Conta', how='left')
posicao.rename(columns={"IR":"Soma de IR", "IOF":"Soma de IOF"}, inplace=True)
posicao.drop("ESCRITÓRIO", axis=1, inplace=True)

In [ ]:
setores = pd.read_excel(r"C:\Scripts\setores_ativos\setores.xlsx")

In [ ]:
posicao['Setor'] = ''
posicao['Subsetor'] = ''
for ativo, emissor in zip(setores['Ativo'], setores['Emissor']):
    try:
        posicao.loc[(posicao['Ativo'] == ativo) | (posicao['Emissor'] == emissor), 'Setor'] = setores[setores['Ativo'] == ativo]['SETOR_ECONÔMICO'].values[0]
        posicao.loc[(posicao['Ativo'] == ativo) | (posicao['Emissor'] == emissor), 'Subsetor'] = setores[setores['Ativo'] == ativo]['SUBSETOR'].values[0]
    except:
        pass

In [ ]:
conexao = Connect.connect_techdb()
posicao.to_sql('posicao', 
               index=False, 
               con=conexao, 
               if_exists='replace', 
               dtype={'CONTA':sqlalchemy.VARCHAR(255)},
               schema="principal")
conexao.close()

<h2>Renda fixa</h2>

In [ ]:
renda_fixa = pd.read_excel(diretorio_base+"%s\\%s\\renda_fixa_coe.xlsx" % (data, horario), header=2)

In [ ]:
for columns in renda_fixa.columns:
    renda_fixa.rename(columns={columns:columns.replace(" (R$)","")}, inplace=True)

In [ ]:
renda_fixa = pd.read_excel(diretorio_base+"%s\\%s\\renda_fixa_coe.xlsx" % (data, horario), header=2)
renda_fixa.rename(columns={"'DL_D_ContaAssessor'[NR_CONTA]":"Conta"}, inplace=True)
renda_fixa['Conta'] = renda_fixa['Conta'].astype(str)
renda_fixa.drop("ID", axis=1, inplace=True)

In [ ]:
conexao = Connect.connect_techdb()
renda_fixa.to_sql('renda_fixa', 
                  index=False, 
                  con=conexao, 
                  if_exists='replace', 
                  dtype={"CONTA":sqlalchemy.VARCHAR(255)},
                  schema="principal")
conexao.close()

<h2>Renda varíavel</h2>

In [ ]:
renda_variavel = pd.read_excel(diretorio_base+"%s\\%s\\renda_variavel.xlsx" % (data, horario), header=2)

In [ ]:
for columns in renda_variavel.columns:
    renda_variavel.rename(columns={columns:columns.replace(" (R$)","")}, inplace=True)

In [ ]:
renda_variavel = pd.read_excel(diretorio_base+"%s\\%s\\renda_variavel.xlsx" % (data, horario), header=2)
renda_variavel.rename(columns={"Data Vencimento":"DATA VENCIMENTO"}, inplace=True)
renda_variavel['DATA VENCIMENTO'] = pd.to_datetime(renda_variavel['DATA VENCIMENTO'], format=("%Y-%m-%d"), errors='coerce')
renda_variavel.rename(columns={"'DL_D_ContaAssessor'[NR_CONTA]":"Conta"}, inplace=True)
renda_variavel['Conta'] = renda_variavel['Conta'].astype(str)
renda_variavel['Valor Bruto Inicial'] = renda_variavel['Preço Médio']*renda_variavel['Quantidade']
renda_variavel['Performance'] = ((renda_variavel['Valor Bruto']-renda_variavel['Valor Bruto Inicial'])/renda_variavel['Valor Bruto Inicial'])*100
renda_variavel.loc[renda_variavel['Performance'] == np.inf, "Performance"] = 0

In [ ]:
conexao = Connect.connect_techdb()
renda_variavel.to_sql('renda_variavel', 
                      index=False, 
                      con=conexao, 
                      if_exists='replace', 
                      dtype={"CONTA":sqlalchemy.VARCHAR(255)},
                      schema="principal")
conexao.close()

<h2>Rentabilidade</h2>

In [ ]:
rentabilidade = pd.read_excel(diretorio_base+"%s\\%s\\rentabilidade.xlsx" % (data, horario), header=2)
rentabilidade['Conta'] = rentabilidade['Conta'].astype(str)

In [ ]:
rentabilidade = \
    rentabilidade.loc[:,["Conta", "Data", 'Profitability mtd',
        'Profitability last 12 months accumulated',
        'Profitability last 6 months accumulated',
        'Profitability last 3 months accumulated', 'Profitability ytd',
        'Modelo de Rentabilidade']]

In [ ]:
conexao = Connect.connect_techdb()
rentabilidade.to_sql('rentabilidade', 
                        index=False, 
                        con=conexao, 
                        if_exists='replace', 
                        dtype={"Conta":sqlalchemy.VARCHAR(255)},
                        schema="principal")
conexao.close()

#### PL Base

In [ ]:
# Hoje
base_hoje = pd.read_excel(diretorio_base+"%s\\%s\\base_btg.xlsx" % (data, horario), header=2)
for columns in base_hoje.columns:
    base_hoje.rename(columns={columns:columns.replace(" (R$)","")}, inplace=True)
base_hoje['Conta'] = base_hoje['Conta'].astype(str)

base.rename(columns={'CONTA':'Conta'}, inplace=True)
base['Conta'] = base['Conta'].astype(str)

In [ ]:
conexao = Connect.connect_techdb()
pl_base_hist = Connect.import_table(conexao, "PL Base")
conexao.close()

In [ ]:
def tratar_base(base):
    base = base[['Assessor', 'Conta', 'PL Total']]
    base.rename(columns={'PL Total':'PL'}, inplace=True)
    return base

In [ ]:
base_hoje = tratar_base(base_hoje)
base_hoje['Mês'] = datetime.today().strftime("%Y-%m-%d")

In [ ]:
pl_base_hist = pl_base_hist[pl_base_hist['Mês'] < datetime.today().strftime("%Y-%m-01")]

In [ ]:
base_hoje.rename(columns={"Conta":"CONTA"}, inplace=True)

In [ ]:
## Concatenar todos os meses
pl_mes_vigente = pd.concat([pl_base_hist,
                            base_hoje], axis=0)

pl_mes_vigente.loc[pl_mes_vigente['Assessor'] == "Murilo Luiz Silva Gino", "Assessor"] = "Fernando Domingues da Silva"

In [ ]:
conexao = Connect.connect_techdb()
pl_offshore_hist = Connect.import_table(conexao, "offshore_adicionar_pl_mes_vigente")
offshore_pl = Connect.import_table(conexao, "pl_offshore")
conexao.close()

In [ ]:
offshore_pl

In [ ]:
pl_offshore_hist = pl_offshore_hist[pl_offshore_hist['Mês'] < datetime.today().strftime("%Y-%m-01")]
pl_offshore_hist['Mês'] = [pd.to_datetime(data).strftime("%Y-%m-%d") for data in pl_offshore_hist['Mês']]
offshore_pl['Mês'] = datetime.today().strftime("%Y-%m-%d")
offshore_pl.rename(columns={"PL Total":"PL", "Conta":"CONTA"}, inplace=True)
offshore_pl.drop("Nome", axis=1, inplace=True)

In [ ]:
adicionar_offshore_pl = pd.concat([offshore_pl, pl_offshore_hist], axis=0)

In [ ]:
conexao = Connect.connect_techdb()
adicionar_offshore_pl.to_sql("offshore_adicionar_pl_mes_vigente", 
                            index=False, 
                            con=conexao, 
                            if_exists='replace',
                            schema="principal")
conexao.close()

In [ ]:
#contas = base['Conta']
#pl_mes_vigente = pl_mes_vigente[pl_mes_vigente['Conta'].isin(contas)]
pl_mes_vigente['PL'] = pl_mes_vigente['PL'].fillna(0)
pl_mes_vigente['Assessor'] = pl_mes_vigente['Assessor'].apply(lambda nome: str(nome).upper())
pl_mes_vigente = pd.concat([pl_mes_vigente, adicionar_offshore_pl], axis=0)
pl_mes_vigente['Mês'] = [pd.to_datetime(data) for data in pl_mes_vigente['Mês']]

In [ ]:
pl_mes_vigente.loc[pl_mes_vigente['Assessor'] == "RODRIGO DE MELLO DELIA", "Assessor"] = "RODRIGO DE MELLO D’ELIA"
pl_mes_vigente.loc[pl_mes_vigente['Assessor'] == "ROSANA PAVANI", "Assessor"] = "ROSANA APARECIDA PAVANI DA SILVA"
pl_mes_vigente.loc[pl_mes_vigente['Assessor'] == "FERNANDO DOMINGUES", "Assessor"] = "FERNANDO DOMINGUES DA SILVA"

In [ ]:
pl_mes_vigente['CONTA'] = pl_mes_vigente['CONTA'].astype(str)

In [ ]:
pl_mes_vigente = pl_mes_vigente.drop_duplicates(subset=["CONTA", "Mês"], keep='first')

In [ ]:
conexao = Connect.connect_techdb()
pl_mes_vigente.to_sql('PL Base', 
                      index=False, 
                      con=conexao, 
                      if_exists='replace',
                      schema="principal")
conexao.close()

In [ ]:
print("Ultima atualização: ", datetime.today())
import winsound
frequency = 2500  # fequência em hertz
duration = 500  # duração. 500 = 0,5s
winsound.Beep(frequency, duration)

In [ ]:
from sqlalchemy import column

horario = [datetime.today().strftime("%d/%m/%Y %H:%M:%S")]
horario_atualizacao = pd.DataFrame(horario, columns=["Horario"])

In [ ]:
horario_atualizacao

In [ ]:
conexao = Connect.connect_techdb()
horario_atualizacao.to_sql(
    "horario_atualizacao_relatorios", 
    index=False, 
    con=conexao, 
    if_exists='replace', 
    schema="principal")
conexao.close()